## Elliptic Curve ElGamal Encryption




#### referecne
- [Elliptic Curve ElGamal Encryption Explained](https://www.youtube.com/watch?v=1hERYQADqrU&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=18)
- [Elliptic Curve ElGamal Encryption in Python From Scratch](https://www.youtube.com/watch?v=062ilU5dOzY&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=18)
- [Elliptic Curve ElGamal Encryption - Sefik Ilkin Serengil](https://sefiks.com/2018/08/21/elliptic-curve-elgamal-encryption/#google_vignette)
- [Elegant Signatrues with Elliptic Curve Cryptography](https://sefiks.com/2018/02/16/elegant-signatures-with-elliptic-curve-cryptography/)
- [Double and Add Method](https://sefiks.com/2016/03/27/double-and-add-method/)
- [Partially Homomorphic Elliptic Curve EIGamal In Python From Scratch](https://www.youtube.com/watch?v=6yN00ttpjSE&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=19)

In [1]:
import random
import hashlib

In [2]:
# y^2 = x^3 + a* x + b

In [3]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
     
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
     
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
     
    is_on_curve((x3, y3), p)
         
    return x3, y3
 
def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
     
def apply_double_and_add_method(G, k, p):
    target_point = G
     
    k_binary = bin(k)[2:] #0b1111111001
     
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
         
        # doubling - always
        target_point = add_points(target_point, target_point, p)
         
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
     
    is_on_curve(target_point, p)
     
    return target_point


In [4]:
# from :https://sefiks.com/2018/02/16/elegant-signatures-with-elliptic-curve-cryptography/
# Secp256k1
a = 0; b = 7
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)


#finite field
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [5]:
is_on_curve(G,p)

In [6]:
tmp_point = G
for i in range(2,21):
    tmp_point = add_points(tmp_point, G, p)
    print(f"{i}G = {tmp_point}")

2G = (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930)
3G = (112711660439710606056748659173929673102114977341539408544630613555209775888121, 25583027980570883691656905877401976406448868254816295069919888960541586679410)
4G = (103388573995635080359749164254216598308788835304023601477803095234286494993683, 37057141145242123013015316630864329550140216928701153669873286428255828810018)
5G = (21505829891763648114329055987619236494102133314575206970830385799158076338148, 98003708678762621233683240503080860129026887322874138805529884920309963580118)
6G = (115780575977492633039504758427830329241728645270042306223540962614150928364886, 78735063515800386211891312544505775871260717697865196436804966483607426560663)
7G = (41948375291644419605210209193538855353224492619856392092318293986323063962044, 48361766907851246668144012348516735800090617714386977531302791340517493990618)
8G = (2126205

In [7]:
apply_double_and_add_method(G= G, k = 20, p= p)

(34773495056115281091786765947597603724784643419904767525769502836017890139287,
 8470533044743364938367028725608288731153024648869546164814808839694950063162)

In [8]:
# Q  = k x G

## Elliptic Curve EIGamal

In [9]:
import random

In [10]:
# alice
ka = random.getrandbits(256) # private of Alice
Qa = apply_double_and_add_method(G= G, k = ka, p = p)

### encryption

In [11]:
def encrypt(m, r):
    s = apply_double_and_add_method(G = G, k = m, p = p)
    
    c1 = apply_double_and_add_method(G = G, k = r, p = p)

    c2 = apply_double_and_add_method(G = Qa, k = r, p = p)

    c2 = add_points(c2, s, p)
    
    return c1, c2

In [12]:
# bob
m = 23061912
r = random.getrandbits(128)

c1 , c2 = encrypt(m, r)

In [13]:
(c1, c2)

((67932942643477861137570749318193179372365035511177219913526477686254759985685,
  96346287051504101160781191221261490297751369518233018073737786391687562671162),
 (54001615288904438499547598191396284522045618649289907913372471121439637096585,
  27689371508052194115809552373168162248070234664727247475655652473513122785713))

### decryption

In [14]:
# s_prime = c2 - ka x c1
# s_privme = c2 + ( ka x -c1)
# (x, y) + (x, -y) = o

In [15]:
c1_prime = (c1[0], ((-1*c1[1]) % p))

In [16]:
s_prime = apply_double_and_add_method(G =c1_prime, k = ka, p = p)
s_prime = add_points(P = c2, Q = s_prime, p = p)

In [17]:
s_prime

(101401888938080707466461718009381950095311426580467620560673978804974259848061,
 81617838766714377738166229284883214383180826546122608693522719057922160918473)

In [19]:
s_real = apply_double_and_add_method(G = G, k = m, p = p)

In [21]:
assert s_real == s_prime

In [ ]:
# S = m x G
# Extracting m from given S and G is hard. Remember ECDLP.

##  partially  homomorphic features

- [Partially Homomorphic Elliptic Curve ElGamal In Python From Scratch](https://www.youtube.com/watch?v=6yN00ttpjSE&list=PLsS_1RYmYQQEun1MTwmvbXurqHIJrFJ0e&index=19)

In [22]:
m1 = 333
r1 =  random.getrandbits(256)


m2 = 777
r2 = random.getrandbits(256)

In [23]:
c1_x, c2_x = encrypt(m1, r1)
c1_y, c2_y = encrypt(m2, r2)

In [24]:
m1_encrypted_plus_m2_encrypted = (
    add_points(
        P = c1_x,
        Q = c1_y,
        p = p),
    add_points(
        P = c2_x,
        Q = c2_y,
        p = p)
)

In [25]:
m1_encrypted_plus_m2_encrypted

((16293082494909202026668116138433544969116815689665562688388021786473793795108,
  62717204492809097912502142116326690081471901525228560025899585945952408546333),
 (100874565268028775236043646930511156629689628123228345795629771907264896671234,
  100128310350871996116828588293706705304972243545185077971605850484135476568690))

In [26]:
encrypt(m1 + m2, r1+ r2)

((16293082494909202026668116138433544969116815689665562688388021786473793795108,
  62717204492809097912502142116326690081471901525228560025899585945952408546333),
 (100874565268028775236043646930511156629689628123228345795629771907264896671234,
  100128310350871996116828588293706705304972243545185077971605850484135476568690))

In [27]:
assert encrypt(m1 + m2, r1+ r2) == m1_encrypted_plus_m2_encrypted